In [3]:
import urllib.request
import pandas as pd
import bs4 as bs
from datetime import datetime

url = 'https://www.rivm.nl/coronavirus-kaart-van-nederland-per-gemeente'
response = urllib.request.urlopen(url).read()
soup = bs.BeautifulSoup(response)
csv_input = soup.find('div', id='csvData').get_text()

import csv
myFile = open('RIVM_data/'+ datetime.today().strftime("%m-%d-%Y %H-%M") + '.csv', 'w')
with myFile:
   writer = csv.writer(myFile)
   writer.writerows([x.split(';') for x in csv_input.split('\n')])

In [4]:
import pandas as pd
pd.read_csv('RIVM_data/test.csv',encoding = 'cp1252')

,,,,Unnamed: 0
Gemnr,Gemeente,Aantal,BevAant,Aantal per 100.000 inwoners
-1,* postcode ontbreekt bij 84 pt en 1 woont in buitenland,99,NaN,NaN
518,'s-Gravenhage,21,537833,3.9
796,'s-Hertogenbosch,30,154205,19.5
1680,Aa en Hunze,0,25386,0
358,Aalsmeer,3,31728,9.5
197,Aalten,0,27011,0
59,Achtkarspelen,0,27852,0
482,Alblasserdam,1,20069,5
613,Albrandswaard,4,25271,15.8


In [5]:
df = pd.DataFrame([x.split(';') for x in csv_input.split('\n')])
df.columns = df.iloc[1]
df = df[3:]
df['statnaam']=df['Gemeente']
df['BevAant'] = pd.to_numeric(df['BevAant'])
df['Aantal'] = pd.to_numeric(df['Aantal'])
df.rename(columns={'Aantal per 100.000 inwoners':'Aantal per 100000 inwoners'},inplace=True)
df.head()

In [22]:
import pandas as pd
pd.read_csv('RIVM_data/test.csv',encoding = 'cp1252')

,,,,Unnamed: 0
Gemnr,Gemeente,Aantal,BevAant,Aantal per 100.000 inwoners
-1,* postcode ontbreekt bij 84 pt en 1 woont in buitenland,99,NaN,NaN
518,'s-Gravenhage,21,537833,3.9
796,'s-Hertogenbosch,30,154205,19.5
1680,Aa en Hunze,0,25386,0
358,Aalsmeer,3,31728,9.5
197,Aalten,0,27011,0
59,Achtkarspelen,0,27852,0
482,Alblasserdam,1,20069,5
613,Albrandswaard,4,25271,15.8


In [11]:
import altair as alt
from vega_datasets import data

url1 = 'https://cartomap.github.io/nl/rd/gemeente_2020.topojson'
url2 = 'https://cartomap.github.io/nl/wgs84/gemeente_2020.topojson' 

gemeentes = alt.topo_feature(url2, 'gemeente_2020')

from IPython import display
import ipywidgets as widgets
import altair as alt
import pandas as pd
alt.renderers.enable('html')
w = widgets.IntSlider()


def generate_chart(col):
  # print(col)

  chart = alt.Chart(gemeentes).mark_geoshape().properties(
    width=300,
    height=200
    
    ).encode(
    color=alt.Color(col+':Q',scale=alt.Scale(type='symlog')),
    tooltip=['properties.statnaam:N', col+':Q']
    ).transform_lookup(
    lookup='properties.statnaam',
    from_=alt.LookupData(df, 'Gemeente', [col]),
    default= '0')
  return chart

output = widgets.Output()
with output:
    display.display(generate_chart('Aantal')|generate_chart('Aantal'))
output;

alt.HConcatChart(...)